In [3]:
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
import torch.nn as nn

In [4]:
#Download the data
data_path='../data-unversioned/p1ch7/'
cifar10=datasets.CIFAR10(data_path,train=True,download=True,transform=transforms.ToTensor())
cifar10_val=datasets.CIFAR10(data_path,train=False,download=True,transform=transforms.ToTensor())


In [5]:
imgs=torch.stack([img_t for img_t,_ in cifar10],dim=3) #returns 3*32*32*50000 tensor
mean=imgs.view(3,-1).mean(dim=1) # 3*32*32 image becomes 3*1024 then mean is calculated.
std=imgs.view(3,-1).std(dim=1)  # Same for std.

In [6]:
#Normalized dataset
transformed_cifar10=datasets.CIFAR10(data_path,train=True,download=False,transform=transforms.Compose
                                    ([
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)
                                    ]))

In [7]:
#Keeping only birds and planes.
#training data
label_map={0:0,2:1}
class_names=['airplane','bird']
cifar2=[(img,label_map[label]) for img,label in cifar10 if label in [0,2]]
#validation data
cifar2_val=[(img,label_map[label]) for img,label in cifar10_val if label in [0,2]]

In [10]:
#input features (image to 1D vector) 3*32*32=3072
n_in=3072
#output features (bird or airplane)
n_out=2
model=nn.Sequential(nn.Linear(n_in,1024),
                    nn.Tanh(),
                    nn.Linear(1024,512),
                    nn.Tanh(),
                    nn.Linear(512, 128),
                    nn.Tanh(),
                    nn.Linear(128, n_out))
learning_rate=1e-2
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn=nn.CrossEntropyLoss()
train_loader=torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)
n_epochs=100
for epoch in range(n_epochs):
    for imgs,labels in train_loader:
        batch_size=imgs.shape[0]
        outputs=model(imgs.view(batch_size,-1))
        loss=loss_fn(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {float(loss)}")


In [1]:
validation_loader=torch.utils.data.DataLoader(cifar2_val,batch_size=64,shuffle=False)
correct=0
total=0
with torch.no_grad():
    for imgs,labels in validation_loader:
        batch_size=imgs.shape[0]
        outputs=model(imgs.view(batch_size,-1))
        _,predicted=torch.max(outputs,dim=1)
        total+=labels.shape[0]
        correct+=int((predicted==labels).sum())
print(f"({correct/total})")

NameError: name 'torch' is not defined